## Moedas: Atomoação web para atualização dos valores

Neste trabalho iremos buscar na internet, de forma automática, a cotação de diversas moedas, tais como dólar, euro, libra e etc

A ideia é conferir se o valor está abaixo do preço ideal de compra imposto por uma empresa fictícia. Caso esteja, iremos marcar como uma opção de compra para a equipe de operações.

Usaremos o selenium, uma biblioteca que permite controlar o funcionamento de uma página e a interação com ela de forma automática. 

Base de dados: google.com

In [60]:
from selenium import webdriver
import pandas as pd

In [61]:
# Abrindo o nabegador

navegador = webdriver.Chrome()
navegador.get("https://www.google.com")

In [62]:
# Importando a base de dados

tabela = pd.read_excel("moedas.xlsx")

display(tabela)

,Moeda,Preço Ideal,Preço Atual,Comprar
0,Dólar,4.90,NaN,NaN
1,Euro,5.43,NaN,NaN
2,Peso,0.05,NaN,NaN
3,Libra,6.01,NaN,NaN


In [63]:
# Buscando as cotações

for linha in tabela.index:
    
    #Entrando no site que contém as cotações
    moeda = tabela.loc[linha, "Moeda"]
    moeda = moeda.replace("ó", "o")
    link = f"https://www.google.com/search?q=cotacao+{moeda}&oq=cotacao+peso&aqs=chrome.0.69i59.3424j0j9&sourceid=chrome&ie=UTF-8"
    print(link)
    navegador.get(link)
        
    # Pegando e transformando a cotação em valor para o Python
    cotacao = navegador.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
    cotacao = cotacao.replace(",", ".")
    cotacao = float(cotacao)
    cotacao = round(cotacao, 2)
    print(cotacao)
    
    # Preenchendo a coluna Preço Atual
    tabela.loc[linha, "Preço Atual"] = cotacao


https://www.google.com/search?q=cotacao+Dolar&oq=cotacao+peso&aqs=chrome.0.69i59.3424j0j9&sourceid=chrome&ie=UTF-8
4.99
https://www.google.com/search?q=cotacao+Euro&oq=cotacao+peso&aqs=chrome.0.69i59.3424j0j9&sourceid=chrome&ie=UTF-8
5.4
https://www.google.com/search?q=cotacao+Peso&oq=cotacao+peso&aqs=chrome.0.69i59.3424j0j9&sourceid=chrome&ie=UTF-8
0.02
https://www.google.com/search?q=cotacao+Libra&oq=cotacao+peso&aqs=chrome.0.69i59.3424j0j9&sourceid=chrome&ie=UTF-8
6.22


In [64]:
display(tabela)

,Moeda,Preço Ideal,Preço Atual,Comprar
0,Dólar,4.90,4.99,NaN
1,Euro,5.43,5.40,NaN
2,Peso,0.05,0.02,NaN
3,Libra,6.01,6.22,NaN


In [65]:
# Preenchendo a coluna Comprar

tabela["Comprar"] = tabela["Preço Ideal"] >= tabela["Preço Atual"]

In [66]:
display(tabela)

,Moeda,Preço Ideal,Preço Atual,Comprar
0,Dólar,4.90,4.99,False
1,Euro,5.43,5.40,True
2,Peso,0.05,0.02,True
3,Libra,6.01,6.22,False


In [67]:
# Fechando o navegador

navegador.quit()

# Exportando a base de dados atualizada

tabela.to_excel("moedas_atualizado.xlsx", index=False)